In [1]:
import cv2
from PIL import Image
import numpy as np

# Function to generate a binary mask of an image using edge detection
def generate_binary_mask(image, threshold1=80, threshold2=200):
    
    # grayscale image
    # convert image to numpy array
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(image, threshold1, threshold2)
    
    # Convert edges to binary image (0 or 255)
    binary_image = cv2.threshold(edges, 127, 255, cv2.THRESH_BINARY)[1]
    
    return binary_image

In [2]:
from PIL import Image
import os
import random

# Function to overlay images with transformations
def overlay_images_with_transformations(part_path, gripper_path, output_path):

    # Load images
    part = Image.open(part_path).convert("RGBA")
    gripper = Image.open(gripper_path).convert("RGBA")

    # Generate binary mask
    part = generate_binary_mask(part)

    # convert part image to RGBA
    part = Image.fromarray(part).convert("RGBA")

    # check if the gripper image is larger than the part image
    if part.width < gripper.width or part.height < gripper.height:
        print("Gripper image is larger than the part image. Exiting...")
        return -1, -1, -1
    
    # the shift_x and shift_y are the values that are used to shift the gripper image
    # the shift value is calculated by placing the gripper image on the top left corner of the part image
    # and then shifting it by the shift value

    # (0, 0) is the top left corner of the image
    start_x = 0 
    start_y = 0
    max_shift_x = -(part.width - gripper.width) // 2  # negative value because we want to shift the gripper to the right
    max_shift_y = -(part.height - gripper.height) // 2 # negative value because we want to shift the gripper down
    # print("Center x: ", start_x, "Center y: ", start_y)
    # print("Max shift x: ", max_shift_x, "Max shift y: ", max_shift_y)

    # Random shift values should be negative only and should be less than part width - gripper width
    shift_x = random.randint(max_shift_x, 0)
    shift_y = random.randint(max_shift_y, 0)
    rotation = random.randint(0, 360)
    # print("Shift x: ", shift_x, "Shift y: ", shift_y, "Rotation: ", rotation)
    shift_x = start_x + max(min(shift_x, -max_shift_x), max_shift_x)
    shift_y = start_y + max(min(shift_y, -max_shift_y), max_shift_y)
    # print("Shift x: ", shift_x, "Shift y: ", shift_y, "Rotation: ", rotation)
    # Transformations to apply
    transformations = [
        {'type': 'rotate', 'value': rotation},
        {'type': 'shift', 'x': shift_x, 'y': shift_y}
    ]


    # Note: We don't resize the images now, but during training we will resize the images to a fixed size
    
    # Apply rotation transformation
    for transform in transformations:
        if transform['type'] == 'rotate':
            gripper = gripper.rotate(transform['value'], expand=True)
    
    # Apply shift transformation
    for transform in transformations:
        if transform['type'] == 'shift':
            shift_x = start_x + max(min(transform['x'], -max_shift_x), max_shift_x) 
            shift_y = start_y + max(min(transform['y'], -max_shift_y), max_shift_y) 
            gripper = gripper.transform(part.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
            print("Shift x: ", shift_x, "Shift y: ", shift_y, "Rotation: ", rotation)
    
    # store the shift values and rotation as string in the output path
    output_path = output_path[:-4] + "_x_" + str(shift_x) + "_y_" + str(shift_y) + "_rotation_" + str(rotation) + ".png"

    # Overlay images
    combined = Image.alpha_composite(part, gripper)
    combined.save(output_path)

    return shift_x, shift_y, rotation

In [3]:
# Paths to your images
gripper_file = "data/train_images/Grippers/Gripper_4.png"
part_folder = "data/train_images/Metal_sheets/"
#part_folder = "data/train_images/Test/"
output_folder = "data/train_images/Train/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [4]:
import os

# Function to parse through a directory and apply a specified function to each .png file
def parse_directory_and_apply_function(root_directory):
    for subdir, _, files in os.walk(root_directory):
        for file in files:
            if file.endswith(".png"):
                # combine folder name and part name
                part = part_folder + file
                output_file = output_folder + file


                x, y, rotation = overlay_images_with_transformations(part, gripper_file, output_file)

parse_directory_and_apply_function(part_folder)

/tmp/ipykernel_79632/2072731233.py:62: DeprecationWarning: AFFINE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transform.AFFINE instead.
  gripper = gripper.transform(part.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))


Shift x:  -1 Shift y:  -63 Rotation:  259
Shift x:  -11 Shift y:  -15 Rotation:  6
Shift x:  -6 Shift y:  -41 Rotation:  174
Shift x:  -19 Shift y:  -29 Rotation:  325
Gripper image is larger than the part image. Exiting...
Shift x:  -1 Shift y:  -29 Rotation:  220
Shift x:  -16 Shift y:  -20 Rotation:  153
Gripper image is larger than the part image. Exiting...
Shift x:  -68 Shift y:  -15 Rotation:  73
Shift x:  -6 Shift y:  -9 Rotation:  149
Shift x:  -10 Shift y:  -85 Rotation:  206
Shift x:  -18 Shift y:  -10 Rotation:  259
Shift x:  -5 Shift y:  -18 Rotation:  235
Shift x:  -2 Shift y:  -9 Rotation:  42
Shift x:  -29 Shift y:  -49 Rotation:  131
Shift x:  -42 Shift y:  -21 Rotation:  78
Gripper image is larger than the part image. Exiting...
Shift x:  -23 Shift y:  -51 Rotation:  270
Gripper image is larger than the part image. Exiting...
Shift x:  -50 Shift y:  -65 Rotation:  348
Shift x:  -3 Shift y:  -25 Rotation:  268
Shift x:  0 Shift y:  -20 Rotation:  27
Gripper image is la